In [345]:
# importar librerías
import pandas as pd
import openpyxl

In [352]:
# bases de datos en excel
df_cronicos = pd.read_excel("data/cronicos.xlsx")
df_egresos = pd.read_excel("data/egresos.xlsx")
df_usuarios = pd.read_csv("data/usuarios.csv", low_memory=False)

In [364]:
# Seleccionar los ID de los usuarios de la columna "CICLO VITAL" que son jóvenes
jovenes = df_usuarios[df_usuarios["CICLO_VITAL"] == "Juventud"]

# Solo unir los ID de los jovenes con los ID de los cronicos
jovenes_cronicos = jovenes.merge(df_cronicos, on="NRODOC", how="inner").merge(
    df_egresos, on="NRODOC", how="inner"
)

# Filtrar datos solo de PGP y eliminar columnas innecesarias
jovenes_cronicos = jovenes_cronicos[
    (jovenes_cronicos["EPS VALIDADA"] != "NUEVA EPS S.A.")
    & (jovenes_cronicos["EPS VALIDADA"] != "NO AFILIADO PARTICULAR")
]

# Eliminar filas
jovenes_cronicos = jovenes_cronicos.dropna(subset=["EPS VALIDADA"])

jovenes_cronicos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 0 to 8223
Columns: 370 entries, MES_x to PROFESIONAL ESPECIALIDAD GRD
dtypes: datetime64[ns](3), float64(39), int64(17), object(311)
memory usage: 18.3+ MB


In [365]:
# ver datos null en jovenes_cronicos en porcentaje con el total (solo mostrar los mayores a 0)
nulos = jovenes_cronicos.isnull().sum() / len(jovenes_cronicos) * 100

# Borrar nulos mayores o iguales a 50% (por criterio)
nulos = nulos[nulos >= 50]
jovenes_cronicos = jovenes_cronicos.drop(nulos.index, axis=1)

In [366]:
# Borrar columnas que no se van a usar mencionadas arriba
jovenes_cronicos = jovenes_cronicos.drop(
    [
        "Peso",
        "Tipo",
        "Talla",
        "IMC",
        "YEAR",
        "MES",
        "SERVICIO HABILITADO COD",
        "TIPO IDENTIFICACION",
        "FECHA NACIMIENTO_y",
        "REGIMEN AFILIACION",
        "MODALIDAD CONTRATO",
        "NRO CAMA ANTERIOR",
        "NRO CAMA",
        "SERVICIO EGRESO",
        "DEMORA SALIDA CLINICA (DIAS)",
        "CAUSA BASICA MUERTE",
        "CAUSA BASICA CATEGORIA",
        "CAUSA BASICA CAPITULO",
        "PerÃ­metro Muslo",
        "PerÃ­metro Cintura",
        "Pliegue Triceps",
        "Pliegue Abdomen",
        "Pliegue Muslo",
        "Sumatoria Pliegues",
        "PresiÃ³n Arterial SistÃ³lica",
        "Fecha creatinina1",
    ],
    axis=1,
)


# Eliminar columnas que mencionadas arriba
jovenes_cronicos = jovenes_cronicos.drop(
    [
        "MES_x",
        "YEAR_x",
        "QUINQUENIO",
        "YEAR_y",
        "MES_y",
        "FECHA NACIMIENTO_x",
        "MUNICIPIO",
        "DEPARTAMENTO",
        "Fecha creatinina2",
        "Basciloscopia",
        "FECHA FARMACIA",
        "FECHA CAMILLERO",
        "FECHA ÃšLTIMA CLASE FUNCIONAL",
        "FECHA PRIMERA CLASE FUNCIONAL",
        "FECHA ENFERMERIA",
        "FECHA RECAUDO",
        "FECHA ASEO",
        "FECHA FACTURACION AUDIFARMA",
        "Fecha PrÃ³ximo Control",
        "Tipo control",
        "Tiene prÃ³ximo control",
        "Fecha trigliceridos",
        "Ãšlcera de pie diabÃ©tico",  # Pocos datos
        "Fecha microalbuminuria",
        "Fecha colesterol total",
        "Fecha HDL",
        "FECHA INICIO AL PGP",
        "Control HTA",  # La varibale "tiene riesgo Tiene riesgo de tener HTA" es mejor
        "Fecha glicemia",
        "Meses de diferencia entre TFG",  # Pocas categorias y se concentra en una
        "AnÃ¡lisis y conducta a seguir",  # Texto
        "Cambio de TFG",  # Pocas categorias y se concentra en una
        "Insuficiencia CardÃ­aca",  #
        "Diabetes Mellitus",  # Pocas categorias y se concentra en una
        "ValvulopatÃ­a",  # Pocas categorias y se concentra en una
        "Control Diabetes",  # Pocas categorias y se concentra en una
        "Tiene HTA",
        "Tiene EPOC",
        "Cuantos cigarrillos dÃ­a",
        "AÃ±os de Consumo",
        "DXPRINCIPAL EGRESO COD",
        "DX RELACIONADO1 COD",
        "DX RELACIONADO2 COD",
        "DX RELACIONADO3 COD",
        "DX PRINCIPAL EGRESO CATEGORIA COD",
        "DX PRINCIPAL EGRESO CAPITULO COD",
        "FECHA INGRESO SERVICIO",
    ],
    axis=1,
)

# ver datos null en jovenes_cronicos en porcentaje con el total (solo mostrar los mayores a 0)
nulos = jovenes_cronicos.isnull().sum() / len(jovenes_cronicos) * 100

In [367]:
jovenes_cronicos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 0 to 8223
Data columns (total 87 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   NRODOC                                     6464 non-null   int64         
 1   SEXO                                       6464 non-null   object        
 2   EDAD                                       6464 non-null   float64       
 3   BARRIO                                     6078 non-null   object        
 4   FECHA INICIO AL PGP                        5759 non-null   object        
 5   PRIMERA CLASE FUNCIONAL                    4683 non-null   object        
 6   ÃšLTIMA CLASE FUNCIONAL                    5333 non-null   object        
 7   CICLO_VITAL                                6464 non-null   object        
 8   Fecha diligenciamiento                     6464 non-null   datetime64[ns]
 9   Hora                    

In [362]:
# For con value_counts() para ver los valores de las columnas
for col in jovenes_cronicos.columns:
    print(col)
    print("----------------------------------")
    print(jovenes_cronicos[col].value_counts())
    print()

NRODOC
----------------------------------
NRODOC
312214    2793
308002     756
312667     540
312920     432
307997     420
310335     320
307227     272
308001     168
311380     155
307508     126
312933     126
312217      57
310336      56
313556      56
312976      54
313285      49
310369      28
313428      24
312918      21
314091       8
313416       3
Name: count, dtype: int64

SEXO
----------------------------------
SEXO
F    3522
M    2942
Name: count, dtype: int64

EDAD
----------------------------------
EDAD
27.0    1882
25.0    1109
26.0     939
20.0     803
28.0     696
24.0     590
21.0     392
23.0      40
19.0      13
Name: count, dtype: int64

BARRIO
----------------------------------
BARRIO
ROBLEDO EL DIAMANTE    2646
VILLA HERMOSA           702
BELLO - EL TRAPICHE     540
ESPIRITU SANTO          432
LA FLORESTA             420
SAN JAVIER              272
BELLO LAS VEGAS         180
GUAYABAL                168
BELLO                   140
SAN CRISTOBAL           126